# by @sol2man2

# 14.1 순수 함수적 변이 가능 상태

## 참조 투명서오가 순수성의 정의

프로그램 p에서 나타나는 모든 표현식 e를 e의 평가 결과로 대치할 때 p에 아무런 영향을 미치지 않는 경우 참조에 투명하다라고 표현한다.

표현식 f(x)가 참조에 투명한 모든 x에 대해 참조에 투명하면 함수f 는 순수하다.

* swap, partition, qs는 참조 투명하지 않다.
* 외부에서는 내부에 변이를 모른다.
* 전체적으로 quicksort는 참조 투명하다.

In [28]:
quicksort(List(3,6,2,8,5,9,4,1,7))

: 

* quicksort는 제대로, 효율적이기 위해서는 자료를 제자리 이동해야 한다.
* 스칼라는 지역에서 생성된 자료는 안전하게 변이할 수 있다.
* 함수적 프로그래밍에서는 순수 함수적 구현을 더 선호하지만 이는 구성요소를 제대로 만들기 쉽고 함수들을 합성하기 쉽다는 이유이다.

# 14.2 부수 효과를 지역 범위로 한정하는 자료 형식

부수 효과의 범위 적용을 느슨하게 추론하는 것이 잘못된 것은 아니지만 경우에 따라서 형식 시스템을 이용하여 효과의 범위 적용을 **강제**하는 것이 바람직할 수도 있다.

13장의 IO를 사용할 수도 있지만 적합하지 않다. IO[List[Int]]를 반환해도 된다. 하지만 새로운 타입을 만들어 보자.

## 14.2.1 범위 있는 변이를 위한 작은 언어

가장 자연스러운 접근 방식은 변이 가능 상태를 서술하는 작은 언어를 만드는 것이다.

상태를 읽고 쓰는 것에는 기존의 State[S,A]모나드로도 가능하다. 이 모나드는 S => (A, S)의 형식이다.

하지만 지금 필요한 것은 제자리에서 변이하는 모나드가 필요하다.

S형식의 토큰을 넘겨주면 호출된 함수가 S로 표시된 자료를 변이할 **권한**을 갖는다.

다음의 두 불변식이 성립해야 코드가 컴파일 된다.

* 함수 안에 변이 가능 객체에 대한 참조가 있다고 할 때, 함수 외부에서는 그 참조에 대한 변이를 전혀 관측할 수 없다.
* 변이 가능 객체는 그것이 생성된 범위 밖에서는 전혀 관측할 수 없다.

quicksort는 첫 불변식을 만족한다. 함수는 배열을 변이하지만, 그 배열에 대한 참조가 함수 안에만 있으므로 함수 밖에서는 그러한 변이를 전혀 관측하지 못한다.

### 형식에 의한 변이 한정의 또 다른 용례

??

In [30]:
sealed trait ST[S,A] { self =>
    protected def run(s: S): (A,S)
    def map[B](f: A => B): ST[S,B] = new ST[S,B] {
        def run(s: S) = {
            val (a, s1) = self.run(s)
            (f(a), s1)
        }
    }
    def flatMap[B](f: A => ST[S,B]): ST[S,B] = new ST[S,B] {
        def run(s: S) = {
            val (a, s1) = self.run(s)
            f(a).run(s1)
        }
    }
}

object ST {
    def apply[S,A](a: => A) = {
        lazy val memo = a
        new ST[S,A] {
            def run(s: S) = (memo, s)
        }
    }
}

defined trait ST
defined object ST

run 메소드를 protected로 한 것은 S가 상태의 변이 능력을 나타내는데 그 변이가 범위를 탈출해서는 안되기 때문이다.

## 14.2.2 변이 가능 참조의 대수

ST 모나드의 첫 응용 예제는 변이 가능 참조를 서술하는 작은 언어이다. 이 언어는 몇가지 기본 수단들로 이루어진 조합기 라이브러리의 형태를 취한다.

* 새 변이 가능 칸을 할당한는 명령
* 변이 가능 칸에 값을 쓰는(기록하는) 명령
* 변이 가능 칸을 읽는 명령

In [31]:
sealed trait STRef[S,A] {
    protected var cell: A
    def read: ST[S,A] = ST(cell)
    def write(a: A): ST[S,Unit] = new ST[S,Unit] {
        def run(s: S) = {
            cell = a
            ((), s)
        }
    }
}

object STRef {
    def apply[S,A](a: A): ST[S, STRef[S,A]] = ST(new STRef[S,A] {
        var cell = a
    })
}

defined trait STRef
defined object STRef

* 메모리를 읽고 쓰는 STRef의 메소드는 ST 동작만 돌려주므로 순수 함수다. S가 변이되는 형식이 아나며 S형식의 값을 실제로 사용하는 일은 전혀 없다.
* apply를 호출해서 ST 동작 중 하나를 실제로 실행핳려면 S형식의 값이 필요하다.
* STRef는 sealed되어 있다. 인스턴스를 임의로 생성 못하고 오브젝트를 통해서만 생성가능하다.

In [49]:
val stref =
for {
    r1 <- STRef[Nothing,Int](1)
    r2 <- STRef[Nothing,Int](1)
    x <- r1.read
    y <- r2.read
    _ <- r1.write(y+1)
    _ <- r2.write(x+1)
    a <- r1.read
    b <- r2.read
} yield (a,b)

stref: ST[Nothing, (Int, Int)] = cmd20$$user$ST$$anon$2@53e7bdb4

In [50]:
println(stref.run(Unit))
// 권한 땜에 실행하지 못한다.

: 

## 14.2.3 변이 가능 상태 동작의 실행

우리의 계획은

* 어떤 지역 변이 가능 상태를 할당
* 그것을 변이해서 어떤 과제를 수행
* 그 변이 가능 상태를 폐기하는 계산을 ST를 이용해서 구축

이는 변이 가능 상태가 지여 범위 안에 비공개로 존재하기 때문이다.

근데 우리가 원하는 건 그러한 지역 범위 한정을 보장받는 것이다.

예를 들어 변이 가능 var를 담은 STRef가 있다고 할 때 ST를 STRef 바깥으로 추출하는 것이 불가능함을 스칼라의 형식 시스템이 보장해 주어야 한다.

그렇다면 ST 동작을 안전하게 실행하는 방법은 뭘까? 

첫째로 안전하게 실행할 수 있는 동작과 그렇지 않은 동작을 구분해야 한다.

* ST[S, STRef[S, Int]] 안전하게 실행할 수 없음
* ST[S, Int] 완전히 안전하게 실행할 수 있음

STRef는 S에 관여하지만 Int에는 관여하지 않는다.

ST[S, STRef[S,A]]는 실행을 금지시켜야 한다. 금지시키지 않으면 STRef가 노출될 수 있더,

실행하기에 안전한 ST동작들 다시말해 S에 대해 다형적인 동작들을 대표하는 새 특질을 도입하자.

In [33]:
trait RunnableST[A] {
    def apply[S]: ST[S,A]
}

defined trait RunnableST

RunnableST[A] 형식의 값은 형식 S를 받아서 ST[S,A] 형식의 값을 산출하는 함수이다.

In [34]:
val p = new RunnableST[(Int, Int)] {
    def apply[S] = for {
        r1 <- STRef(1)
        r2 <- STRef(2)
        x <- r1.read
        y <- r2.read
        _ <- r1.write(y+1)
        _ <- r2.write(x+1)
        a <- r1.read
        b <- r2.read
    } yield (a,b)
}

p: AnyRef with RunnableST[(Int, Int)] = cmd24$$user$$anonfun$1$$anon$1@202d241

In [53]:
object ST {
    def apply[S,A](a: => A) = {
        lazy val memo = a
        new ST[S,A] {
            def run(s: S) = (memo, s)
        }
    }
    
    def runST[A](st: RunnableST[A]): A =
        st.apply[Unit].run(())._1
}

: 

In [54]:
val p = new RunnableST[(Int, Int)] {
    def apply[S] = for {
        r1 <- STRef(1)
        r2 <- STRef(2)
        x <- r1.read
        y <- r2.read
        _ <- r1.write(y+1)
        _ <- r2.write(x+1)
        a <- r1.read
        b <- r2.read
    } yield (a,b)
}

p: AnyRef with RunnableST[(Int, Int)] = cmd30$$user$$anonfun$1$$anon$1@19ce2953

In [56]:
val r = ST.runST(p)

: 

runST(p)는 내부적으로 변이 가능 상태를 사용하지만, 부수 효과를 내지는 않는다. 다른 표현식들이 볼 때 이 표현식은 그냥 보통의 정수 쌍일 뿐이다. 이 표현식은 항상 같은 정수 쌍을 돌려주며 그 외의 일은 하지 않는다.

그런데 이것이 가장 중요한 부분은 아니다. 좀 더 중요한 점은 변이 가능 참조를 돌려 주려 하는 프로그램은 실행할 수 없다는 점이다. 노출된 STRef를 돌려주는 RunnableST를 생성하는 것이 불가능하다..

In [57]:
new RunnableST[STRef[Nothing,Int]] {
    def apply[S] = STRef(1)
}

: 

이 예에서는 임의의 Nothing을 선택했지만 그 점은 중요치 않음. 요점은 **형식 S가 apply 메소드에 묶이므로 new RunnableST 블록 안에서는 그 형식에 접근할 수 없다는 점**이다.

STRef에는 항상 자신이 속한 ST 동작의 형식 S가 꼬리표로 붙으므로, STRef는 결코 ST동작을 벗어나지 못한다. 그리고 이 점은 다름 아닌 스칼라의 형식 시스템이 보장해 준다. 더 나아가서 ST동작에서 STRef를 얻을 수 없다는 사실은 다음과 같은 명제를 보장한다.

> 만일 STRef를 얻었다면 그 지점은 그 STRef를 생성한 ST동작의 내부이며 따라서 참조를 변이해서 항상 안전하다.

### 와일드 카드 형식 참고 사항

## 14.2.4 변이 가능 배열

변이 가능 참조는 그 자체로는 그리 유용하지 않다. ST 모나드에 대해 훨씬 더 유용하게 사용할 수 있는 것이 변이 가능 배열이다 이번 절에서는 ST모나드 안에서 변이 가능 배열을 조작하기 위한 대수를 정의하고 제자리 quicksort알고르짐을 합성 방식으로 작성한다.

우선 변이 가능 배열의 할당, 읽기, 쓰기를 위한 기본 수단 조합기들이 푤요

In [60]:
sealed abstract class STArray[S,A](implicit manifest: Manifest[A]) {
    protected def value: Array[A]
    def size: ST[S,Int] = ST(value.size)
    def write(i: Int, a: A): ST[S,Unit] = new ST[S,Unit] {
        def run(s: S) = {
            value(i) = a
            ((), s)
        }
    }
    def read(i: Int): ST[S,A] = ST(value(i))
    def freeze: ST[S,List[A]] = ST(value.toList)
}

object STArray {
    def apply[S,A:Manifest](sz: Int, v: A): ST[S, STArray[S,A]] =
        ST(new STArray[S,A] {
        lazy val value = Array.fill(sz)(v)
    })
}

defined class STArray
defined object STArray

스칼라에서 모든 형식 A에 대해 배열을 생성할 수 없음을 주목하자. 배역을 만들려면 암묵적 범위 안에 Manifest[A]가 존재해야 한다. 스칼라 표준 라이브러리는 실제 응용에서 배열에 넣을 만한 대부분의 형식에 대한 매니페스트를 제공한다.

STRef에서 처럼 이 기본 수단들은 항상 STArray를 해당 S형식과 함께 ST동작 안에 담아서 돌려준다. 이후 그 배열에 대한 모든 조작은 같은 형식 S가 꼬리표로 붙은 ST동작이다. 따라서 ST 모나드 밖에서 원본 STAray가 관측되는 일은 없다.

In [61]:
def fromList[S,A:Manifest](xs: List[A]): ST[S, STArray[S,A]] =
    ST(new STArray[S,A] {
        lazy val value = xs.toArray
    })

defined function fromList

## 14.2.5 순수 함수적 제자리 quicksort

In [63]:
def swap[S](i: Int, j: Int): ST[S,Unit] = for {
    x <- read(i)
    y <- read(j)
    _ <- write(i, y)
    _ <- write(j, x)
} yield ()

: 

### exercise 14.2

In [38]:
def noop[S] = ST[S, Unit](())

defined function noop

In [39]:
def partition[S](arr: STArray[S,Int], 
                 n: Int, r: Int, pivot: Int): ST[S,Int] = for {
    vp <- arr.read(pivot)
    _ <- arr.swat(pivot, r)
    j <- STRef(n)
    _ <- (n until r).foldLeft(noop[S])((s,i) => for {
        _ <- s
        vi <- arr.read(i)
        _ <- if(vi < vp) (for {
            vj <- j.read
            _ <- a.swap(i, vj)
            _ <- j.write(vj + 1)
        } yield ()) else noop[S]
    } yield())
    x <- j.read
    _ <- a.swap(x, r)
}

: 

In [40]:
def qs[S](a: STArray[S,Int], n: Int, r: Int): ST[S,Unit] = if(l < r) for {
    pi <- partition(a, n, r, n + (r - n) / 2)
    _ <- qs(a, n, pi - n)
    _ <- qs(a, pi + 1, r)
} yield () else noop[S]

: 

In [41]:
def quicksort(xs: List[Int]): List[Int] =
    if (xs.isEmpty) xs else ST.runST(new RunnableST[List[Int]] {
        def apply[S] = for {
            arr <- STArray.fromList(xs)
            size <- arr.size
            _ <- qs(arr, 0, size - 1)
            sorted <- arr.freeze
        } yield sorted
    })

: 

이 구현에서 보듯이 ST 모나드를 이용하면 주어진 자료를 변이하긴 하지만 그래도 순수한 함수들을 작성할 수 있다. 그리고 구성요소들을 안전하지 않은 방식으로 조합하는 일은 스칼라의 형식 시스템 자체가 방지를 해준다.

# 4.3 순수성은 문맥에 의존한다.

In [24]:
case class Foo(s: String)
val b = Foo("hello") == Foo("hello")
val c = Foo("hello") eq Foo("hello")

defined class Foo
b: Boolean = true
c: Boolean = false

In [25]:
val x = Foo("hello")
val d = x eq x

x: Foo = Foo("hello")
d: Boolean = true

## 14.3.1 부수 효과로 간주되는 것은 무엇인가?

In [64]:
def timesTwo(x: Int) = {
    if (x < 0) println("Got a negative number")
    x * 2
}

defined function timesTwo

# 14.4 요약

이번 장에서는 참조 투명성이 뜻하는 두 가지 의미를 논의했다.

이번 장에서는 자료의 변이가 지역 범위를 벗어나지 않는다면 문제가 되지 않음을 보았다. 처음에는 변이 가능 상태가 순수 함수와 호환되지 않는다고 생각했겠지만 이번 장에서 보았듯이 내부적으로 지역 상태를 변이하면서도 순수한 인터페이스를 가진 구성요소들을 작성할 수 있으며 스칼라의 형식 시스템을 이용해서 순수성을 보장할 수 있다.

또한 이번 장에서는 부수 효과로 간주되는 것이 실제로는 프로그래머 또는 언어 설계자의 한 선택이라는 점도 논의 했다. 함수의 순수성을 논의할 때 우리는 어떠한 규정적인 문맥, 즉 두 가지 것이 서로 같다는 것이 무슨 뜻이고 프로그램의 의미라는 것이 무엇인지 그리고 프로그램에 의미를 부여할 때 어떤 효과들을 고려할 것인가를 규정하는 하나의 문맥을 이미 선택해 두고 논의를 진행한다.

# Thank you so much

# Q & A